# データの整形

## カテゴリデータの処理

```
データ
 L 数値データ
 L カテゴリデータ
    L 順序特徴量(ordinal) ... 服のサイズの S, M, L, XL など順序がある
    L 名義特徴量(nominal) ... 色の 赤, 青, 緑 など順序がない
```

データセットには、カテゴリデータ（S,M,Lや赤,青,緑など）の特徴量が含まれている場合があります。
さらに、カテゴリデータには、順序特徴量と名義特徴量があります

In [22]:
# サンプルデータを生成（色、サイズ、価格、ラベル）
import pandas as pd

df = pd.DataFrame([
            ['green', 'M',  10.1, 'class1'],
            ['red',     'L',   13.5, 'class2'],
            ['blue',   'XL', 15.3, 'class1'],
])
df.columns = ['color', 'size', 'price', 'classlabel']
df

,color,size,price,classlabel
0,green,M,10.1,class1
1,red,L,13.5,class2
2,blue,XL,15.3,class1


## 順序特徴量のエンコーディング

In [23]:
# ディクショナリを作成
size_mapping = { 'M': 1, 'L': 2, 'XL': 3 }

# Tシャツのサイズを整数に変換
df['size'] = df['size'].map(size_mapping)
df

# サイズの逆変換
# inv_size_mapping = { v: k for k, v in size_mapping.items()}
# df['inv_size'] = df['size'].map(inv_size_mapping)

,color,size,price,classlabel
0,green,1,10.1,class1
1,red,2,13.5,class2
2,blue,3,15.3,class1


## 名義特徴量のエンコーディング

scikit-learnの多くの学習アルゴリズムは、クラスラベルが整数値としてエンコードされていることを前提としています。そのため、クラスラベルを整数に変換する必要があります。
LabelEncoderを使って整数に変換できます。

In [24]:
# クラスラベルを変換する
from sklearn.preprocessing import LabelEncoder
class_le = LabelEncoder()
y = class_le.fit_transform(df['classlabel'].values)
print('Before Class Label:', df['classlabel'].values)
print('After Class Label:', y)

Before Class Label: ['class1' 'class2' 'class1']
After Class Label: [0 1 0]


In [25]:
# 特徴 color を整数に変換
color_le = LabelEncoder()
X = df[['color', 'size', 'price']].values
X[:, 0] = color_le.fit_transform(X[:, 0])
X

array([[1, 1, 10.1],
       [2, 2, 13.5],
       [0, 3, 15.3]], dtype=object)

In [26]:
# one-hotエンコーディング
# 特徴 color をダミー特徴量(特徴量のユニークな値の数だけ特徴量を展開する)にする
# 0 => [1, 0, 0]
# 1 => [0, 1, 0]
# 2 => [0, 0, 1]
from sklearn.preprocessing import OneHotEncoder
ohe = OneHotEncoder(categorical_features=[0])
ohe.fit_transform(X).toarray()

array([[  0. ,   1. ,   0. ,   1. ,  10.1],
       [  0. ,   0. ,   1. ,   2. ,  13.5],
       [  1. ,   0. ,   0. ,   3. ,  15.3]])

## Pandasの便利なメソッド

In [27]:
# pandas の get_dummiesメソッドを使うと簡単にエンコーディングができるので便利
print('* Before:')
df[['color', 'size', 'price']]

* Before:


,color,size,price
0,green,1,10.1
1,red,2,13.5
2,blue,3,15.3


In [28]:
print('* After:')
pd.get_dummies(df[['color', 'size', 'price']])

* After:


,size,price,color_blue,color_green,color_red
0,1,10.1,0.0,1.0,0.0
1,2,13.5,0.0,0.0,1.0
2,3,15.3,1.0,0.0,0.0
